tutorial -         
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/32296/         
新增特殊字元to stopwords.txt             
stopwords來源: https://github.com/tomlinNTUB/Python-in-5-days/blob/master/10-2%20%E4%B8%AD%E6%96%87%E6%96%B7%E8%A9%9E-%E7%A7%BB%E9%99%A4%E5%81%9C%E7%94%A8%E8%A9%9E.md

In [1]:
num_classes = 4

num_DocsInClass = [9,22,10,14]  #training set

num_AllDocs = sum(num_DocsInClass)

prior_prob = []
for i in range(num_classes):
    prior_prob.append(num_DocsInClass[i] / num_AllDocs)
print(prior_prob)

[0.16363636363636364, 0.4, 0.18181818181818182, 0.2545454545454545]


### Extract terms
jieba

In [2]:
import pandas as pd
from urllib.request import urlopen
import numpy as np
import jieba
import re
import os
import emoji

'''userdict'''
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('./userdict.txt')

'''stopword list'''
stopword_list = []
with open('stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopword_list.append(data)

#term
dictionary = list()  #terms, df
class_TFlist = list()  #(terms, tf in that doc)
class_DFlist = list()  #(terms, tf in that doc)
doc_tokens_list = []
class_tokens_list = []
N = num_AllDocs

for classes in range(num_classes):  #num_classes, 4 
    class_TFdict = dict()
    class_DFdict = dict()
    
    
    '''get the files under the directory'''''''''
    for info in os.listdir(r'./data_posts/'+str(classes+1)): #1,2,3,4      
        doc_has_word = list()  
        post1 = pd.read_csv('data_posts/{}/{}'.format(str(classes+1),info), encoding="utf-8", dtype=str) 
        
        temp_tokens_list = []
        # for each row
        for i in range(post1.shape[0]):  
            text = ''
            try:
                words = post1['post_caption'][i]
                text = text + words + " "   
            except:
                None
                           
            # Remove emoji
            text = emoji.demojize(text)
            # Remove English words & numbers
            #text = re.sub("[A-Za-z0-9\!\%\[\]\,\。\-\#\?\/]", "", text) 
            text = re.sub("[A-Za-z0-9\!\%\[\]\,\。\-\#\?\/\.\_\“\”\、]", "", text) 
            # Remove the redundant spaces
            text = re.sub(r'\s\s+',' ', text)   
            # print(info, text)

            '''Tokenize'''
            tokens_list = jieba.lcut(text, cut_all=False)
            # print(tokens_list)
            
            '''remove stopwords'''
            indx1 = 0
            while indx1 < len(tokens_list):
                if tokens_list[indx1] in stopword_list or tokens_list[indx1]==' ' or tokens_list[indx1]=='\n':
                    del tokens_list[indx1]
                else:
                    if tokens_list[indx1] in class_TFdict:      #term freq of that class
                        class_TFdict[tokens_list[indx1]] += 1
                    else:
                        class_TFdict[tokens_list[indx1]] = 1
                        if tokens_list[indx1] not in dictionary:        
                            dictionary.append(tokens_list[indx1]) 
                    if tokens_list[indx1] not in doc_has_word:
                        if tokens_list[indx1] in class_DFdict:     #(term,一class中有幾篇文件有該term)    
                            class_DFdict[tokens_list[indx1]] += 1
                        else:
                            class_DFdict[tokens_list[indx1]] = 1  
                        doc_has_word.append(tokens_list[indx1])
                    indx1 += 1
            temp_tokens_list.extend(tokens_list)
        doc_tokens_list.append(temp_tokens_list)
    class_tokens_list.append(doc_tokens_list)     
             
    class_TFdict = sorted(class_TFdict.items(), key=lambda d: d[1], reverse = True)
    class_TFlist.append(class_TFdict)     
    class_DFdict = sorted(class_DFdict.items(), key=lambda d: d[1], reverse = True)       
    class_DFlist.append(class_DFdict)   
     
        
dictionary = sorted(dictionary) 
print(len(dictionary))
print(len(class_TFlist))
print(len(class_DFlist))            


Building prefix dict from C:\Users\aduser01\Desktop\IRTM\Final Project\NaiveBayes\dict.txt.big.txt ...
Loading model from cache C:\Users\aduser01\AppData\Local\Temp\jieba.u5d7072edb4d70e1372a6972f753c6c78.cache
Loading model cost 0.981 seconds.
Prefix dict has been built succesfully.


34265
4
4


In [3]:
# number of tokens (以過濾掉stopwords)
count = 0
for i in range(52):  #
    count += len(doc_tokens_list[i])
print(count)

176355


In [4]:
for i in range(num_classes):
    print(class_TFlist[i][:100])

[('吃', 147), ('超', 137), ('最後', 128), ('其實', 123), ('走', 121), ('感覺', 116), ('不要', 113), ('現在', 111), ('照片', 109), ('時間', 107), ('第一次', 107), ('可愛', 104), ('開心', 102), ('快樂', 102), ('拍', 93), ('好像', 91), ('愛', 88), ('老師', 86), ('事', 81), ('應該', 77), ('事情', 76), ('更', 72), ('可能', 71), ('幫', 69), ('玩', 69), ('生活', 68), ('記得', 67), ('完', 66), ('加油', 65), ('前', 65), ('辛苦', 65), ('努力', 63), ('一定', 61), ('不能', 61), ('好吃', 61), ('地方', 60), ('朋友', 59), ('快', 58), ('好多', 58), ('超級', 58), ('每天', 58), ('聽', 58), ('笑', 57), ('一點', 57), ('整個', 57), ('蠻', 57), ('兩個', 56), ('日子', 56), ('問題', 55), ('竟然', 54), ('遇到', 54), ('特別', 54), ('嗚嗚', 53), ('表演', 53), ('比較', 52), ('完全', 52), ('有人', 51), ('跑', 50), ('哈哈哈', 50), ('每個', 49), ('時', 48), ('累', 48), ('結束', 47), ('補', 47), ('同學', 46), ('繼續', 46), ('能夠', 45), ('直接', 45), ('想要', 45), ('那種', 45), ('高中', 44), ('畢業', 44), ('明天', 43), ('根本', 43), ('認真', 42), ('人生', 42), ('小時', 42), ('世界', 41), ('－', 40), ('這張', 40), ('哭', 40), ('真', 39), ('耶', 39), ('話', 38),

In [5]:
'''write dictionary to txt'''
text_file = open( "output_dictionary.txt", "w")
for indx in range(len(dictionary)): 
    try:
        text_file.write(dictionary[indx] + '\n')     # the actual doc_id   
    except:
        None
text_file.close()


'''過濾掉非中文字'''
for i in range(1):    ######## num_classes
    indx = 0
    while indx < len(class_TFlist[i]):
        if class_TFlist[i][indx][0] in dictionary[:389]:
            del class_TFlist[i][indx]
        else:
            indx += 1
    indx = 0
    while indx < len(class_DFlist[i]):
        if class_DFlist[i][indx][0] in dictionary[:389]:
            del class_DFlist[i][indx]
        else:
            indx += 1
            
dictionary

In [6]:
class_DFlist[0][57]

('回來', 8)

## Feature Extraction (top 120 terms in each class)
Likelihood Ratios     
-2log(L(H1) / L(H2))

In [7]:
''' build matrix '''
appear_mtrx = np.zeros((len(dictionary),num_classes))  #class中有某term的文章數量

for class_idx in range(num_classes):
    for term_idx in range(len(class_DFlist[class_idx])): # 726 terms in class 1
        indxxx = dictionary.index(class_DFlist[class_idx][term_idx][0])
        appear_mtrx[indxxx][class_idx] = class_DFlist[class_idx][term_idx][1]  

''' likelyhood ratios '''
likelyhood_ratios = np.zeros((len(dictionary),num_classes))
for term_idx in range(appear_mtrx.shape[0]): # all terms
    for class_idx in range(num_classes):   
        N11 = appear_mtrx[term_idx][class_idx]  #t在c中出現篇數
        N10 = num_DocsInClass[class_idx] - N11
        N01 = sum(appear_mtrx[term_idx][:]) - N11
        N00 = num_AllDocs - num_DocsInClass[class_idx] - N01
        # print(N11 + N01 + N00 + N10)
        likelyhood_ratios[term_idx][class_idx] = -2*np.log10(np.power( (N11+N01)/num_AllDocs, N11 )*
                                                             np.power( 1-(N11+N01)/num_AllDocs, N10 )*
                                                             np.power( (N11+N01)/num_AllDocs, N01 )*
                                                             np.power( 1-(N11+N01)/num_AllDocs, N00 )/
                                                             np.power( N11/(N11+N10), N11 )/
                                                             np.power( 1-N11/(N11+N10), N10 )/
                                                             np.power( N01/(N01+N00), N01 )/
                                                             np.power( 1-N01/(N01+N00), N00 ))
#class_TFlist[0][150][0] # term
#class_TFlist[0][150][1] # class' tf    

''' Feature Selection for Multiple Classifiers '''
'''全部一起選topk個'''
#class_TFlist[0][150][0] # term
#class_TFlist[0][150][1] # class' tf    

#''' Feature Selection for Multiple Classifiers '''
#mean_array = np.zeros(likelyhood_ratios.shape[0])
#for i in range(likelyhood_ratios.shape[0]):
#    mean_array[i] = np.mean(likelyhood_ratios[i][:])
#
#top_k = 150
#top_k_idx = mean_array.argsort()[::-1][0:top_k]


''' chi-square ratios '''
chi_square_ratios = np.zeros((len(dictionary),num_classes))
for term_idx in range(appear_mtrx.shape[0]): # all terms
    for class_idx in range(num_classes):   
        N11 = appear_mtrx[term_idx][class_idx]  #t在c中出現篇數
        N10 = num_DocsInClass[class_idx] - N11
        N01 = sum(appear_mtrx[term_idx][:]) - N11
        N00 = num_AllDocs - num_DocsInClass[class_idx] - N01
        N = N11+N10+N01+N00
        present_ratio = (N11+N10)/N
        on_topic_ratio = (N11+N01)/N
        non_present_ratio = (N01+N00)/N
        not_on_topic_ratio = (N10+N00)/N
        exp1 = N * present_ratio * on_topic_ratio
        exp2 = N * present_ratio * not_on_topic_ratio
        exp3 = N * non_present_ratio * on_topic_ratio
        exp4 = N * non_present_ratio * not_on_topic_ratio
        chi_square_ratios[term_idx][class_idx] = np.power(N11-exp1,2)/exp1 + np.power(N10-exp2,2)/exp2 + np.power(N01-exp3,2)/exp3 + np.power(N00-exp4,2)/exp4
# print(len(likelyhood_ratios))
# print(len(chi_square_ratios),chi_square_ratios)



'''each class 選topk個，再合併(remove duplicate)'''
top_k = 150
top_k_idx = []
each_class_top = []
for class_idx in range(num_classes):
    mean_array = likelyhood_ratios[:,class_idx]
    #mean_array = chi_square_ratios[:,class_idx]
    print(mean_array)
    class_top = mean_array.argsort()[::-1][0:top_k] 
    
    #print(class_top[:20])
    
    temp_top = []
    for origin_idx in class_top:
        temp_top.append(dictionary[origin_idx])
    print(temp_top)
    each_class_top.append(temp_top)
    
    top_k_idx.extend( class_top )

    
top_k_idx = list(set(top_k_idx))    # remove duplicate
print(len(top_k_idx))
top_k = len(top_k_idx)

[0.31676868 0.15677551 0.15677551 ... 0.15677551 0.77961693 0.31676868]
['太好', '茶', '回', '蛋糕', '普通', '網美', '獲得', '順帶', '拍出', '小型', '共識', '放手', '流星雨', '後院', '服儀', '有名', '縫', '酸奶', '貓貓', '澎湖', '談論', '歡迎', '便宜', '新', '推', '懶得', '隨便', '手機', '地獄', '背影', '星星', '床上', '滑', '居然', '搭', '學會', '這件', '揍', '組織', '地鐵', '計較', '受控', '外國人', '臺南', '木頭', '發生', '自然', '出門', '再度', '停', '泡', '早點', '愉快', '過了', '原來', '日期', '很會', '值', '抹', '優秀', '變好', '討厭', '發文', '早', '穿', '後悔', '瞎妹', '個位', '整得', '昏倒', '上升', '顆', '愛著', '衛生', '還加', '當了', '坦白', '臘腸狗', '頭還', '海水浴場', '禪寺', '頭才', '人給', '塌', '永和', '人想', '上船', '忍者', '還走', '放上', '呀呀', '搗蛋', '大得', '血檢', '缺一', '孬', '人體', '仙女', '更讓人', '點數', '蘿蔔絲', '畫個', '紀', '麻痺', '海島', '兩行', '禮服', '卡士', '面子', '碩', '七美', '多話', '瓶蓋', '蟬聯', '夠用', '豢養', '多顧慮', '珍重再見', '象山', '書籤', '八分', '夜貓', '豐盛', '夜行', '標的', '掩埋', '糖餅', '耳背', '紀律', '揶揄', '最快', '以此', '約翰', '緬甸', '杏仁', '教壞', '郊遊', '函', '教主', '第一支', '防備', '貴族', '升起', '潛水', '大吉', '燈塔', '伏特加', '安捏', '休息時間', '海市蜃樓']
[ 0.03682469  0.44904836  0.807

In [8]:
#top_k_idx = sorted(top_k_idx)
fea_extr_dictionary = []

for origin_idx in top_k_idx:
    fea_extr_dictionary.append(dictionary[origin_idx])
    #fea_extr_dictionary.append((dictionary[origin_idx], origin_idx))  #原本的index
print(fea_extr_dictionary)

['緩步', '從沒', '緬甸', '練個', '還想', '共識', '還我', '鞋子', '共鳴', '暖', '眼', '史', '多多', '練舞', '縫', '音樂', '氣質', '再度', '拿到', '睡著', '睡覺', '多話', '總覺', '心力', '順利', '順帶', '瞎妹', '多顧慮', '還能', '心情', '按摩', '夜市', '還買', '還走', '更讓人', '永和', '知足', '爸爸', '夜行', '夜貓', '合照', '求救', '夠', '心裡', '書籤', '對話', '夠用', '缺一', '走過', '蘿蔔絲', '人想', '決定', '決賽', '研究', '同學', '砲', '頭才', '破冰', '冷氣', '置信', '最大', '忍者', '捷運', '掃', '最快', '小型', '頭還', '小女孩', '掌聲', '人給', '沒事', '大吉', '顆', '沒人', '人群', '特別', '超好', '蛋糕', '小屁孩', '美國', '排斥', '碩', '沒好', '人體', '沒少', '願意', '小時候', '向前', '郊遊', '最難', '沒想', '會員', '大得', '介紹', '蟬聯', '狂', '會想', '出門', '一日遊', '群組', '接力', '吵', '函', '蠻', '越來越少', '吹', '仙女', '控', '猛', '血檢', '會點', '推', '性感', '呀呀', '猜', '翹', '行事', '社長', '小隊', '沙拉', '掩埋', '以此', '有名', '少', '趕快', '酸奶', '揍', '油', '少少的', '提', '老人', '味道', '衛生', '足跡', '獲得', '一職', '大軍', '飽', '伏特加', '飽飽', '和平', '衣服', '休息時間', '泡', '衣褲', '泡麵', '福利社', '一路', '咖啡', '跟上', '表情', '表演', '禪寺', '表達', '禮服', '衰', '考完', '秀', '揪', '重大', '咻', '考慮', '泰勒', '泰國', '屁股', '重新', '天冷', '揶揄', '居然', 

## Word Cloud 
#### 問卷資料

In [9]:
#for i in range(num_classes):
#    print(class_TFlist[i][:100])

def WordCloud(terms, saveplot=True, name=""):
    import matplotlib.pyplot as plt
    from wordcloud import WordCloud
    wc = WordCloud(font_path='C:\\Windows\\Fonts\\mingliu.ttc', 
               background_color="black",
               prefer_horizontal=0.5,
               max_words = 200)
    plt.figure(dpi = 500)
    wc.generate(" ".join(terms))
    plt.imshow(wc)
    plt.axis("off")
    if(saveplot):
        plt.savefig(name+"_plot.jpg")

common = ['整個','照片']
for i in range(num_classes):
    lst = []
    length = len(class_TFlist[i])
    for j in range(length):
        if class_TFlist[i][:][j][0] in fea_extr_dictionary and class_TFlist[i][:][j][0] not in common:   # in each_class_top[0]:
            lst.append(class_TFlist[i][:][j][0])
            #print(class_TFlist[i][:][j][0],class_TFlist[i][:][j][1])
    print(lst[:20])
    WordCloud(lst, name="class_{}".format(str(i+1)))



['生活', '蠻', '特別', '表演', '同學', '直接', '真', '新', '穿', '需要', '故事', '文', '心情', '看著', '成發', '陪', '跳', '合照', '不錯', '回']
['傳', '生活', '表演', '文', '真', '蠻', '跳', '直接', '特別', '新', '同學', '盃', '跳舞', '陪', '成發', '願意', '合照', '睡覺', '故事', '決定']
['決定', '直接', '心情', '生活', '室友', '表演', '當初', '特別', '蠻', '鞋子', '文', '慢慢', '意外', '看著', '日', '這件', '新', '穿', '目標', '少']
['生活', '特別', '博物館', '文', '故事', '新', '歷史', '想起', '表演', '城市', '包容', '展', '發生', '不夠', '同學', '地鐵', '旅行', '直接', '需要', '心情']


## Training: Naive Bayes
#### "condprob" matrix

In [10]:
condprob = np.zeros((top_k, num_classes))
top_condprob_lst = []

for class_idx in range(num_classes):  
    T_ct = list()  
    termOfClass = list()
    for temp_i in range(len(class_TFlist[class_idx])):
        termOfClass.append(class_TFlist[class_idx][temp_i][0])
        
    for term in fea_extr_dictionary: # all terms
        if term in termOfClass:
            idxInClass = termOfClass.index(term)
            T_ct.append( class_TFlist[class_idx][idxInClass][1])    
        else:
            T_ct.append(0)
    # print(len(T_ct))
    totalWordsInClass = np.sum(T_ct)
    
    for i in range(len(fea_extr_dictionary)): # all terms
        condprob[i][class_idx] = (T_ct[i]+1)/(totalWordsInClass+top_k)
        
    top_condprob = condprob[:,class_idx].argsort()[::-1][:70] 
    top_condprob_lst.append(top_condprob)
    for ttop in top_condprob:
        print(fea_extr_dictionary[ttop])
    print("\n")

照片
生活
蠻
整個
特別
表演
同學
直接
真
新
穿
需要
文
故事
心情
看著
成發
陪
跳
合照
不錯
願意
回
發生
手機
過了
發文
窩
哈哈哈哈
想起
跳舞
後悔
慢慢
進步
搭
決定
舒服
幹話
原來
隨便
日
學會
吵
幫忙
夜市
好開心
惹
便宜
舞
蛋糕
幹嘛
女舞
擔心
放棄
真正
半夜
難
攝影師
旅行
方式
還能
濕
獲得
衣服
出門
少
討厭
自然
暖
偷偷


照片
傳
生活
表演
文
真
蠻
跳
直接
特別
新
整個
同學
盃
跳舞
陪
成發
願意
合照
睡覺
故事
決定
需要
手機
哈哈哈哈
回
幫忙
當初
喝
搭
打球
看著
日
目標
心情
嗨
舞
擔心
順利
心裡
幹嘛
好開心
穿
討厭
後悔
小隊
發生
舞台
原來
支持
幹話
包容
方式
考完
少
發文
衣服
瀏海
過了
聖誕節
不錯
很爽
走過
真正
學會
最大
趕快
進步
意外
出門


照片
決定
整個
直接
心情
生活
室友
表演
當初
特別
蠻
鞋子
文
慢慢
意外
看著
這件
日
穿
新
目標
少
合照
拿到
故事
願意
舒服
真
還能
發生
心裡
手機
隔天
放棄
順利
不錯
真正
抽獎
留下
回
隨便
睡覺
老人
小隊
哥哥
最大
同學
紀錄
自動
對話
幫忙
臭
搭
總覺
喝
再度
肯定
包容
跳
介紹
興奮
衣服
表達
想起
裝備
這也
旅行
盃
踩
進步


生活
特別
照片
博物館
故事
文
整個
新
歷史
想起
表演
城市
包容
展
發生
地鐵
不夠
旅行
同學
直接
需要
心情
美國
討厭
重新
蠻
一路
願意
小時候
看著
最大
手機
回
陪
過了
慢慢
好開心
決定
留下
泡
發文
搭
教育
心裡
偷偷
夠
當初
複雜
女人
放棄
走過
音樂
少
要說
原來
鏡頭
拿到
真正
山
做到
放學
秘密
優秀
砲
支持
隔天
哈哈哈哈
日
早就
早




## validation

In [11]:
'''每篇文件中出現幾次(含重複) each_class_top 字詞'''
for i, each_doc in enumerate(doc_tokens_list):
    print("doc  "+str(i+1))
    count = 0
    for word in each_doc:
        if word in each_class_top[0]:
            count += 1
    print(count)
    count = 0
    for word in each_doc:
        if word in each_class_top[1]:
            count += 1
    print(count)
    count = 0
    for word in each_doc:
        if word in each_class_top[2]:
            count += 1
    print(count)
    count = 0
    for word in each_doc:
        if word in each_class_top[3]:
            count += 1
    print(count)
# num_DocsInClass = [9,22,12,15]  

doc  1
28
8
17
48
doc  2
76
2
29
82
doc  3
206
55
197
727
doc  4
138
28
107
267
doc  5
89
8
43
112
doc  6
84
11
47
150
doc  7
51
3
18
70
doc  8
86
6
45
168
doc  9
159
34
186
484
doc  10
33
86
70
287
doc  11
0
1
0
6
doc  12
53
80
79
289
doc  13
0
0
3
2
doc  14
7
13
17
25
doc  15
119
124
135
383
doc  16
365
440
540
1776
doc  17
0
0
0
1
doc  18
10
29
19
62
doc  19
182
166
247
803
doc  20
3
15
11
50
doc  21
2
9
3
13
doc  22
185
202
355
822
doc  23
8
21
17
65
doc  24
7
16
16
36
doc  25
14
28
23
91
doc  26
5
7
15
22
doc  27
63
100
151
335
doc  28
58
113
130
248
doc  29
10
16
10
32
doc  30
2
22
21
83
doc  31
6
17
9
45
doc  32
17
6
28
102
doc  33
17
8
15
89
doc  34
14
9
17
80
doc  35
0
0
0
6
doc  36
5
4
10
65
doc  37
7
0
2
9
doc  38
86
27
69
425
doc  39
8
3
26
116
doc  40
1
0
1
12
doc  41
25
4
37
115
doc  42
1
0
2
0
doc  43
0
2
5
0
doc  44
11
4
36
38
doc  45
2
2
7
12
doc  46
20
8
27
14
doc  47
3
4
5
8
doc  48
0
0
0
0
doc  49
14
13
41
36
doc  50
0
1
3
0
doc  51
54
48
96
164
doc  52
5
1
13
12


In [12]:
import csv
for info in os.listdir(r'./validation_posts/'): #info
    tokens = list()                   ################
    score = np.zeros(num_classes)
    post1 = pd.read_csv('validation_posts/{}'.format(info), encoding="utf-8", dtype=str)        
    # for each row
    for i in range(post1.shape[0]):  
        text = ''
        try:
            words = post1['post_caption'][i]
            text = text + words + " "   
        except:
            None
        text = emoji.demojize(text)
        text = re.sub("[A-Za-z0-9\!\%\[\]\,\。\-\#\?\/\.\_\“\”\、]", "", text)
        text = re.sub(r'\s\s+',' ', text)   
        tokens_list = jieba.lcut(text, cut_all=False)
        indx1 = 0
        for word in tokens_list:
            if word not in stopword_list or tokens_list[indx1]!=' ':
                tokens.append(word)  

    '''classification'''
    # 4 classes, count 4 probabilities P(c|d) ~> P(c)*P(d|c)
    for classes in range(num_classes):
        score[classes] = np.log10(prior_prob[classes])
        for word in tokens: 
            if word in fea_extr_dictionary:     #else:ignore
                idxx = fea_extr_dictionary.index(word)
                score[classes] += np.log10(condprob[idxx][classes]) 

    # print(score)
    predict_class = np.argmax(score)
    # print('predicted class ID of doc.', doc_id,':', predict_class + 1)  #ID: 1~13

    with open('validation_output.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)          # 建立CSV檔寫入器
        writer.writerow( [info[:-4], info[:1], predict_class+1] )


#### F1 score

In [13]:
from sklearn.metrics import f1_score, precision_score, recall_score

post = pd.read_csv('validation compare.csv', encoding="utf-8", dtype=str)        
y_true = post['classification']    
y_pred1 = post['chi-square']    
y_pred2 = post['likelyhood']    

#chi-square
f1 = f1_score( y_true, y_pred1, average='macro' )
p = precision_score(y_true, y_pred1, average='macro')
r = recall_score(y_true, y_pred1, average='macro')
print(f1, p, r)

#likelyhood
f1 = f1_score( y_true, y_pred2, average='macro' )
p = precision_score(y_true, y_pred2, average='macro')
r = recall_score(y_true, y_pred2, average='macro')
print(f1, p, r)

0.7595238095238096 0.7916666666666666 0.7499999999999999
0.8374999999999999 0.9 0.8333333333333333


## Classification (Testing)

In [37]:
import csv
wordDict = dict()
''' Extract Tokens From Doc'''   
# get the files under the directory
test_cat = ['athlete','celebrity','politician','writer','youtuber']
for cat in test_cat:
    for info in os.listdir(r'./test_posts/{}/'.format(cat)): #info
        tokens = list()                   ################
        score = np.zeros(num_classes)
        post1 = pd.read_csv('test_posts/{}/{}'.format(cat,info), encoding="utf-8", dtype=str)        
        # for each row
        for i in range(post1.shape[0]):  
            text = ''
            try:
                words = post1['post_caption'][i]
                text = text + words + " "   
            except:
                None
            text = emoji.demojize(text)
            text = re.sub("[A-Za-z0-9\!\%\[\]\,\。\-\#\?\/\.\_\“\”\、]", "", text)
            text = re.sub(r'\s\s+',' ', text)   
            tokens_list = jieba.lcut(text, cut_all=False)
            indx1 = 0
            for word in tokens_list:
                if word not in stopword_list or tokens_list[indx1]!=' ':
                    tokens.append(word)  
          
            # tokens_list中包含top_condprob_lst[class_index]的字詞 &次數
            for wordInx in top_condprob_lst[0]:  #第一類別
                if fea_extr_dictionary[wordInx] in tokens_list:
                    if fea_extr_dictionary[wordInx] not in wordDict:
                        wordDict[fea_extr_dictionary[wordInx]] = 1
                    else:
                        wordDict[fea_extr_dictionary[wordInx]] += 1

        '''classification'''
        # 4 classes, count 4 probabilities P(c|d) ~> P(c)*P(d|c)
        for classes in range(num_classes):
            score[classes] = np.log10(prior_prob[classes])
            for word in tokens: 
                if word in fea_extr_dictionary:     #else:ignore
                    idxx = fea_extr_dictionary.index(word)
                    score[classes] += np.log10(condprob[idxx][classes]) 

        # print(score)
        predict_class = np.argmax(score)
        # print('predicted class ID of doc.', doc_id,':', predict_class + 1)  #ID: 1~13

        with open('test_output.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)          # 建立CSV檔寫入器
            writer.writerow( [info[:-4], predict_class+1] )


In [39]:
wordDict

{'願意': 433,
 '新': 809,
 '衣服': 212,
 '生活': 803,
 '放棄': 229,
 '真正': 342,
 '決定': 349,
 '故事': 589,
 '心情': 280,
 '舒服': 123,
 '出門': 170,
 '討厭': 135,
 '需要': 696,
 '直接': 404,
 '幫忙': 172,
 '方式': 308,
 '難': 139,
 '蠻': 135,
 '發生': 360,
 '日': 509,
 '進步': 101,
 '還能': 161,
 '整個': 275,
 '陪': 395,
 '原來': 324,
 '慢慢': 225,
 '少': 156,
 '穿': 324,
 '學會': 194,
 '旅行': 215,
 '真': 404,
 '蛋糕': 86,
 '手機': 264,
 '惹': 73,
 '自然': 101,
 '回': 280,
 '獲得': 171,
 '攝影師': 77,
 '不錯': 166,
 '表演': 103,
 '同學': 205,
 '搭': 219,
 '隨便': 75,
 '特別': 543,
 '合照': 161,
 '照片': 566,
 '看著': 432,
 '幹嘛': 161,
 '偷偷': 168,
 '好開心': 92,
 '便宜': 41,
 '哈哈哈哈': 116,
 '發文': 82,
 '吵': 38,
 '後悔': 66,
 '過了': 112,
 '半夜': 54,
 '濕': 53,
 '擔心': 202,
 '想起': 367,
 '跳舞': 24,
 '跳': 125,
 '暖': 61,
 '文': 118,
 '舞': 16,
 '夜市': 45,
 '窩': 35,
 '幹話': 15,
 '成發': 2}